## Proyecto 1

Empezamos importando nuestras librerías

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb

Creamos la funcipón para la carga de datos. Aquí creamos nuestro dataframe, mezclamos su contenido y separamos nuestros datasets de entranamiento y y pruebas.

In [18]:
def cargaDatos():
    df = pd.DataFrame(np.load('proyecto_training_data.npy'), columns= ['SalePrice', 'OverallQual', '1stFlrSF', 'TotRmsAbvGrd', 'YearBuilt', 'LotFrontage'])
    df= df.sample(frac=1, ignore_index=True)
    filas = len(df)
    slicing = int(filas*0.8)
    df_entre = df[:slicing]
    df_prueba = df[slicing:]

    return df, df_entre, df_prueba

Creamos la función para el análisis exploratorio de datos

In [147]:
def analisis():
    analisisExp = cargaDatos()[1]
    analisisExp = analisisExp.describe().loc[['mean','max', 'min', 'std'],:]
    analisisExp = analisisExp.rename(index={'mean': 'Media','max':'Máximo', 'min':'Mínimo', 'std': 'Desviación estándar'})

    rango = pd.DataFrame({
            'SalePrice': [analisisExp['SalePrice'].max() - analisisExp['SalePrice'].min()],
            'OverallQual': [analisisExp['OverallQual'].max() - analisisExp['OverallQual'].min()],
            '1stFlrSF': [analisisExp['1stFlrSF'].max() - analisisExp['1stFlrSF'].min()],
            'TotRmsAbvGrd': [analisisExp['TotRmsAbvGrd'].max() - analisisExp['TotRmsAbvGrd'].min()],
            'YearBuilt': [analisisExp['YearBuilt'].max() - analisisExp['YearBuilt'].min()],
            'LotFrontage': [analisisExp['LotFrontage'].max() - analisisExp['LotFrontage'].min()]},
            index= ['Rango'])

    analisisExp = pd.concat([analisisExp, rango])
    analisisExp = analisisExp.loc[['Media','Máximo', 'Mínimo', 'Rango', 'Desviación estándar'],:]
    return analisisExp


In [148]:
analisis()

,SalePrice,OverallQual,1stFlrSF,TotRmsAbvGrd,YearBuilt,LotFrontage
Media,179484.005137,6.087329,1160.608733,6.491438,1971.598459,69.641588
Máximo,755000.000000,10.000000,4692.000000,14.000000,2010.000000,313.000000
Mínimo,35311.000000,1.000000,334.000000,2.000000,1872.000000,21.000000
Rango,719689.000000,9.000000,4358.000000,12.380201,1979.862658,292.000000
Desviación estándar,77251.278291,1.396253,393.333357,1.619799,30.137342,24.345793
